In [1]:
import os

os.environ['http_proxy'] = 'http://127.0.0.1:26001'
os.environ['https_proxy'] = 'http://127.0.0.1:26001'

In [5]:
from fastai.vision.all import *

path = untar_data(URLs.IMAGENETTE)

In [8]:
dblock = DataBlock(
    blocks=[ImageBlock, CategoryBlock],
    get_items=get_image_files,
    get_y=parent_label,
    item_tfms=[Resize(460)],
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)
dls = dblock.dataloaders(path, bs=64)

In [9]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.602695,2.303988,0.378267,02:35
1,1.213578,1.834208,0.508962,02:25
2,0.929070,1.494063,0.585138,02:25
3,0.709780,0.782421,0.766617,02:27
4,0.576382,0.590687,0.816281,02:26


In [11]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [12]:
x.mean(dim=[0, 2, 3]), x.std(dim=[0, 2, 3])

(TensorImage([0.4560, 0.4431, 0.4104], device='cuda:0'),
 TensorImage([0.2822, 0.2799, 0.2988], device='cuda:0'))

In [13]:
def get_dls(bs, size):
    dblock = DataBlock(
        blocks=[ImageBlock, CategoryBlock],
        get_items=get_image_files,
        get_y=parent_label,
        item_tfms=[Resize(460)],
        batch_tfms=[*aug_transforms(size=224, min_scale=0.75),
                    Normalize.from_stats(*imagenet_stats)]
    )
    return dblock.dataloaders(path,bs=bs)

dls = get_dls(64, 224)
x, y = dls.one_batch()
x.mean(dim=[0, 2, 3]), x.std(dim=[0, 2, 3])

(TensorImage([-0.0942,  0.0198,  0.0557], device='cuda:0'),
 TensorImage([1.2322, 1.2144, 1.2854], device='cuda:0'))

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

In [14]:
# Progressive Resizing
dls = get_dls(128,128)
learn = Learner(dls, xresnet50(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.895241,3.331595,0.357730,02:29
1,1.328547,1.409644,0.574683,02:26
2,0.984881,0.927441,0.708738,02:26
3,0.777402,0.680828,0.792756,02:26


In [ ]:
learn.dls = get_dls(64,224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.841134,1.019947,0.707618,02:27


epoch,train_loss,valid_loss,accuracy,time


In [ ]:
# Test Time Augmmentation
preds, targs = learn.tta()
accuracy(preds, targs).item()

In [ ]:
# Mixup
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, cbs=Mixup)

learn.fit_one_cycle(5, 3e-3)


In [ ]:
# Label Smoothing
model = xresnet50()

learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)

learn.fit_one_cycle(5, 3e-3)